In [ ]:
# 📌 Step 1: Install & Import
!pip install cohere datasets tqdm

import cohere
from datasets import load_dataset, get_dataset_config_names
import json
import os
import time
from tqdm import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0

In [ ]:
# 🔐 Step 2: Set Up Cohere Client
client = cohere.ClientV2("")

In [ ]:
# 🔐 Step 3: Set Up the folder
from google.colab import drive
drive.mount('/content/drive')

# Navigate to your folder (change the path to match your Google Drive structure)
%cd /content/drive/MyDrive/CodeClarity

In [ ]:
# 🔐 Step 3: Set Up the generation
import cohere
import os


# Ensure summaries directory exists (in current working directory)
output_dir = os.path.join(os.getcwd(), "summaries")
os.makedirs(output_dir, exist_ok=True)

test_file_path = os.path.join(output_dir, "test.txt")

try:
    response = client.chat(
        model="command-a-03-2025",
        messages=[
            {"role": "user", "content": "Say hello in Spanish"}
        ]
    )
    output_text = ''.join(part.text for part in response.message.content)
    print("API test succeeded. Response:")
    print(output_text)

    # Write to summaries/test.txt
    with open(test_file_path, "w", encoding="utf-8") as f:
        f.write(output_text + "\n")
    print(f"Output written to: {test_file_path}")

except Exception as e:
    print("Cohere API test failed:")
    print(e)


API test succeeded. Response:
¡Hola!
Output written to: /content/drive/MyDrive/CodeClarity/summaries/test.txt


In [ ]:
# Required Libraries
import cohere
from datasets import load_dataset, get_dataset_config_names
import json
import os
import time
from tqdm import tqdm

# Resolve current script location
script_dir = os.getcwd()

# Define Natural Languages (from multilingual study)
natural_languages = [
    "Spanish", "Mandarin Chinese", "Arabic", "Swahili", "Yoruba",
    "Tamil", "Hindi", "Portuguese", "Filipino", "French"
]

# Programming Languages from CodeSearchNet
code_languages = get_dataset_config_names("code_search_net")
code_languages = [lang for lang in code_languages if lang != "all"]

# Create summaries directory relative to script
output_dir = os.path.join(script_dir, "summaries")
os.makedirs(output_dir, exist_ok=True)

# Error log file in script directory
error_log_path = os.path.join(script_dir, "errors.log")
if os.path.exists(error_log_path):
    os.remove(error_log_path)

# Set sample size
MAX_SAMPLES = 1  # Adjust as needed

# Summary Generation Loop
SPLIT = "validation"

for code_lang in code_languages:
    print(f"\nLanguage: {code_lang}")
    dataset = load_dataset("code_search_net", code_lang, split=SPLIT, trust_remote_code=True)
    dataset = dataset.shuffle(seed=42).select(range(min(MAX_SAMPLES, len(dataset))))

    for natural_lang in natural_languages:
        print(f"  Summarizing in: {natural_lang}")

        filename = os.path.join(output_dir, f"summary_{code_lang}_{natural_lang.lower().replace(' ', '_')}.jsonl")
        with open(filename, "w", encoding="utf-8") as f:
            for i, example in enumerate(tqdm(dataset, desc=f"{code_lang} → {natural_lang}", unit="fn")):
                code_snippet = example.get("func_code_string")
                docstring = example.get("func_documentation_string")
                if not code_snippet:
                    continue

                messages = [
                    {
                        "role": "system",
                        "content": (
                            "You are helping someone who doesn't know how to code. "
                            "Explain what the function does in simple terms."
                        )
                    },
                    {
                        "role": "user",
                        "content": f"Can you explain what this function does in {natural_lang}?\n\n{code_snippet}"
                    }
                ]

                summary = None
                for attempt in range(3):
                    try:
                        response = client.chat(
                            model="command-a-03-2025",
                            messages=messages,
                            temperature=0.2
                        )
                        summary = ''.join(part.text for part in response.message.content)
                        break
                    except Exception as e:
                        error_msg = f"[{code_lang} | {natural_lang} | Example {i}] Attempt {attempt + 1} failed: {e}"
                        print(error_msg)
                        with open(error_log_path, "a", encoding="utf-8") as err_log:
                            err_log.write(error_msg + "\n")
                        time.sleep(2)

                record = {
                    "code": code_snippet,
                    "docstring": docstring,
                    "summary": summary if summary else "ERROR: Failed to generate summary"
                }

                f.write(json.dumps(record, ensure_ascii=False) + "\n")

        print(f"Saved: {filename}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

code_search_net.py:   0%|          | 0.00/8.44k [00:00<?, ?B/s]


Language: java


java.zip:   0%|          | 0.00/1.06G [00:00<?, ?B/s]

...

Saved: /content/drive/MyDrive/CodeClarity/summaries/summary_php_portuguese.jsonl
  Summarizing in: Filipino


php → Filipino: 100%|██████████| 1/1 [00:12<00:00, 12.01s/fn]


Saved: /content/drive/MyDrive/CodeClarity/summaries/summary_php_filipino.jsonl
  Summarizing in: French


php → French: 100%|██████████| 1/1 [00:10<00:00, 10.92s/fn]

Saved: /content/drive/MyDrive/CodeClarity/summaries/summary_php_french.jsonl
